Okay, let's start over after we've understood some maths. 

In [653]:
# Imports:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Data Setup:
X = np.load('data/X-data.npy')
y = np.load('data/y-data.npy')

np.random.seed(0)

In [654]:
# Abstract Class Definitions: 
class Layer():
    def __init__(self): 
        self.inputs = None  # The inputs into this layer. 
        self.outputs = None # The ouputs of this layer. 
        
    # Forward propagation method.
    def forward(self, inputs):
        pass
    
    # Backward propagation method.
    def backward(self):
        pass
    
class Activation():
    def activate():
        pass
    
    def prime():
        pass
    
class Loss():
    def calculate_loss():
        pass

In [655]:
class ReLU(Activation):
    def activate(self, x): 
        return np.maximum(x, 0.0)
    
    def derivative(self, x):
        return (x > 0) * 1  # * 1 to return a number.
    
class Sigmoid(Activation):
    def activate(self, x):
        return 1 / (1 + np.exp(-x))
    
    def derivative(self, x):
        s = self.activate(x)
        return s * (1 - s)

In [656]:
class CrossEntropyLoss():
    def calculate_loss(self, y_true, y_pred):
        return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))
    
    def derivative(self, y_true, y_pred): 
        r = ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)
        return r

In [657]:
# Dense (fully connected) Layer Class:
class Dense(Layer): 
    def __init__(self, input_size, output_size, activation_function='relu', name='unnamed'):
        self.name = name
        self.weights = np.random.randn(output_size, input_size)
        self.biases = np.zeros((output_size, 1))
        self.outputs = None
        
        if activation_function == 'relu':
            self.activation = ReLU()
        elif activation_function == 'sigmoid':
            self.activation = Sigmoid()
        else:
            self.activation = ReLU()  # Default to ReLU activation function.
        
    def print_weights(self):
        print('Weights:\n', pd.DataFrame(self.weights))
        
    def print_biases(self):
        print('Biases:\n', pd.DataFrame(self.biases))
    
    def forward(self, inputs):
        self.inputs = inputs
        self.outputs = self.activation.activate(np.dot(self.weights, inputs) + self.biases)
        return self.outputs
    
    def backward(self, gradient_vector, learning_rate):
        # We can skip updating self.outputs because they will be changed in the next iteration.
        print('weights', self.weights.shape)
        print('biases', self.biases.shape)
        print('gradient vec', gradient_vector.shape)
        print('inputs', self.inputs.shape)
        self.weights -= learning_rate * np.dot(gradient_vector, self.inputs.T)
        self.biases -= learning_rate * gradient_vector
        return np.dot(self.weights.T, gradient_vector)

In [658]:
class Network:
    def __init__(self, layers, loss_function='cross_entropy'):
        self.layers = layers
        self.output = None
        
        if loss_function == 'cross_entropy':
            self.loss = CrossEntropyLoss()
        else:
            self.loss = CrossEntropyLoss()  # Default to cross entropy loss. 
    
    def train(self, X_train, y_train, number_epochs, learning_rate=0.1):
        for epoch in range(number_epochs):
            error = 0
            
            for x, y in zip(X_train, y_train):
                # Process the forward pass. This goes through every layer.
                self.predict(x)
                
                # Calculate the error after the forward pass. 
                error += self.loss.calculate_loss(y, self.output)

                # Calculate the derivative of the loss in respect to the output.
                dL_dA = self.loss.derivative(y, self.output)  # (1, 1)
                # Shape is the number of neurons of the last layer x 1. 
                
                # Calculate the derivative of the activation function in respect to the weighted sum
                dA_dZ = self.layers[1].activation.derivative(self.layers[1].outputs)  # (1, 1)
                
                # The derivative of the weighted sum in respect to the weights
                dZ_dW = self.layers[1].inputs  # (2, 1)
                
                # The derivative of the loss in respect to the weights, transpose to fit the weights
                dL_dW_L1 = np.multiply(np.multiply(dL_dA, dA_dZ), dZ_dW).T  # (1, 2) or (outputs, inputs)
                
                # Update the weights
                new_weights = self.layers[1].weights - learning_rate * dL_dW_L1
                
                # For the first layer                
                # Calculate the derivative of the activation function in respect to the weighted sum
                delta_L1 = np.multiply(dL_dA, dA_dZ).T
                print('delta_l1', delta_L1, delta_L1.shape)
                weights_L1 = self.layers[1].weights
                print('weights_l1', weights_L1, weights_L1.shape)
                
                step_1 = np.dot(delta_L1, weights_L1)
                print('step 1:', step_1, step_1.shape)
                
                dA_dZ_L0 = self.layers[0].activation.derivative(self.layers[0].outputs)  # (2, 1)
                dA_dZ_L0 = dA_dZ_L0.T  # (1, 2)
                print('da_dz_L0', dA_dZ_L0, dA_dZ_L0.shape) 
                
                step_2 = np.multiply(step_1, dA_dZ_L0)  # (1, 2)
                print('step 2:', step_2, step_2.shape) 
                
                inputs_L0 = self.layers[0].inputs
                print('inputs', inputs_L0.shape, inputs_L0.shape)
                
                weights_L0 = self.layers[0].weights
                print('weights', weights_L0, weights_L0.shape)
    
    def predict(self, x):
        outputs = x.reshape(21, 1)  # Create a matrix for dot product in forward()
        for layer in self.layers:
            outputs = layer.forward(outputs)
        self.output = outputs
            

For backpropagation, the formula for the very last layer is: 
![last-layer-backprop](https://miro.medium.com/max/828/1*zRDMl-GxVO7qENH5dNrZ-g.png)

In [659]:
np.random.seed(42)  # To keep results consistent.

number_inputs = X.shape[1]
epochs = 1

layers = [
    Dense(number_inputs, 2, activation_function='relu', name='Layer 1'),
    Dense(2, 1, activation_function='sigmoid', name='Layer 2')
]

In [660]:
def print_layer_outputs(network):
    for layer in network.layers:
        print(layer.name, layer.outputs.shape, ':\n', layer.outputs)

In [661]:
X_practice = X[0:2]
X_practice.shape

(2, 21)

In [662]:
network = Network(layers)
network.train(X_practice, y, number_epochs=epochs)
#print_layer_outputs(network)

delta_l1 [[0.46858183]] (1, 1)
weights_l1 [[ 0.49671415 -0.1382643   0.64768854  1.52302986 -0.23415337 -0.23413696
   1.57921282  0.76743473 -0.46947439  0.54256004 -0.46341769 -0.46572975
   0.24196227 -1.91328024 -1.72491783 -0.56228753 -1.01283112  0.31424733
  -0.90802408 -1.4123037   1.46564877]
 [-0.2257763   0.0675282  -1.42474819 -0.54438272  0.11092259 -1.15099358
   0.37569802 -0.60063869 -0.29169375 -0.60170661  1.85227818 -0.01349722
  -1.05771093  0.82254491 -1.22084365  0.2088636  -1.95967012 -1.32818605
   0.19686124  0.73846658  0.17136828]] (2, 21)


ValueError: shapes (1,1) and (2,21) not aligned: 1 (dim 1) != 2 (dim 0)